This notebook accesses the most recent benchmark hazard output (as published on the NCI Data Collections)

In [1]:
%matplotlib inline
from __future__ import print_function
import numpy as np

import Utilities.nctools as nctools
import Utilities.metutils as metutils
from Utilities.maputils import find_index

from PlotInterface.maps import HazardMap, FilledContourMapFigure, saveFigure
from matplotlib.backends.backend_nbagg import FigureCanvasNbAgg as FigureCanvas


from IPython.html.widgets import interact, fixed
from IPython.html import widgets
from IPython.display import Image
import seaborn as sns
sns.set_context('talk')

In [2]:
class PlotUnits(object):

    def __init__(self, units):
        labels = {
            'mps': 'm/s',
            'm/s': 'm/s',
            'mph': 'mi/h',
            'kts': 'kts',
            'kph': 'km/h',
            'kmh': 'km/h'
        }

        levels = {
            'mps': np.arange(30, 101., 5.),
            'm/s': np.arange(30, 101., 5.),
            'mph': np.arange(80, 221., 10.),
            'kts': np.arange(60, 201., 10.),
            'kph': np.arange(80, 361., 20.),
            'kmh': np.arange(80, 361., 20.)
        }

        self.units = units
        self.label = labels[units]
        self.levels = levels[units]

unit_opts = ['m/s', 'mph', 'kts', 'kmh']

In [4]:
url = "http://dapds00.nci.org.au/thredds/dodsC/fj6/TCRM/benchmark/hazard/hazard.nc"
url = "C:/WorkSpace/data/TC/tcpub/hazard/hazard.nc"
ncobj = nctools.ncLoadFile(url)
lon = nctools.ncGetDims(ncobj, 'lon')[:]
lat = nctools.ncGetDims(ncobj, 'lat')[:]
yrs = nctools.ncGetDims(ncobj, 'years')[:]
years = [str(int(y)) for y in yrs]
wspd = nctools.ncGetVar(ncobj, 'wspd')
wspdUnits = wspd.units

try:
    wLower  = nctools.ncGetVar(ncobj, 'wspdlower')
    wUpper = nctools.ncGetVar(ncobj, 'wspdupper')
    ciBounds = True
except KeyError:
    ciBounds = False

Set the map configuration (extents, resolution, projection).

In [5]:
minLon = 100.#min(lon)
maxLon = 160.#max(lon)
minLat = -30.#min(lat)
maxLat = -5.#max(lat)
xlon, ylat = np.meshgrid(lon, lat)

map_kwargs = dict(llcrnrlon=minLon, llcrnrlat=minLat, 
                  urcrnrlon=maxLon, urcrnrlat=maxLat,
                  projection='merc', resolution='i')

In [16]:
def plot_map(year, units):
    pU = PlotUnits(units)
    idx = years.index(year)
    data = wspd[idx, :, :]
    fig = HazardMap()
    title = "{0}-year ARI wind speed".format(year)
    cbarlabel = "Wind speed ({0})".format(pU.label)
    levels = pU.levels
    #fig.add(metutils.convert(data, wspd.units, pU.units), 
    #         xlon, ylat, title, levels, cbarlabel, map_kwargs)
    #fig.plot()
    fig.plot(metutils.convert(data, wspd.units, pU.units), 
             xlon, ylat, title, levels, cbarlabel, map_kwargs)
    return fig

In [17]:
fig = plot_map('500', 'mps')
fig.set_size_inches(16, 8)
canvas = FigureCanvas(fig)
fig

In [7]:
interact(plot_map, 
         year=widgets.Dropdown(options=years, value=years[1], description="Recurrence interval"),
         units=widgets.Dropdown(options=unit_opts, value=unit_opts[0], description="Units"))